In [1]:
import pandas as pd
from datetime import date,timedelta
import folium
from folium import Marker
from folium.plugins import MarkerCluster
import math
import matplotlib.pyplot as plt
import seaborn as sns

## WARNING
### This dataset is not updated. It belongs to 2021.06.26
### For xlsx extension you must install openpyxl

In [2]:
covid_19trdata=pd.read_excel("./covid19Vaccination.xlsx")

In [3]:
covid_19trdata

,ID,DATE_,SEQID,CITY,CITY2,_1DOSE,_2DOSE,_TOTAL,POPULATION,DIFF_1DOSE,DIFF_2DOSE,DIFF_TOTAL,PREVID
0,438102,2021-06-26 22:41:31,5412,Adana,Adana,784464,355404,1139868,2258718,15,0,15,438021
1,438103,2021-06-26 22:41:31,5412,Adıyaman,Adiyaman,170847,77955,248802,632459,2,0,2,438022
2,438104,2021-06-26 22:41:31,5412,Afyonkarahisar,Afyon,292611,138934,431545,736912,8,0,8,438023
3,438105,2021-06-26 22:41:31,5412,Ağrı,Agri,94807,41146,135953,535435,0,0,0,438024
4,438106,2021-06-26 22:41:31,5412,Aksaray,Aksaray,131889,62603,194492,423011,3,0,3,438025
...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,438178,2021-06-26 22:41:31,5412,Uşak,Usak,169159,81055,250214,369433,6,0,6,438097
77,438179,2021-06-26 22:41:31,5412,Van,Van,236641,91208,327849,1149342,5,0,5,438098
78,438180,2021-06-26 22:41:31,5412,Yalova,Yalova,129888,65920,195808,276050,2,0,2,438099
79,438181,2021-06-26 22:41:31,5412,Yozgat,Yozgat,151825,81924,233749,419095,1,0,1,438100


In [4]:
covid_19trdata["percent_vaccinated"]=covid_19trdata["_TOTAL"]/covid_19trdata["POPULATION"]

In [5]:
citys_coordinates=pd.read_csv("./tr.csv")

In [6]:
citys_coordinates

,city,lat,lng,country,iso2,admin_name,capital,population,population_proper
0,Istanbul,41.0100,28.9603,Turkey,TR,İstanbul,admin,15154000.0,15029231.0
1,Ankara,39.9300,32.8500,Turkey,TR,Ankara,primary,5503985.0,5503985.0
2,İzmir,38.4127,27.1384,Turkey,TR,İzmir,admin,4320519.0,4320519.0
3,Bursa,40.1833,29.0667,Turkey,TR,Bursa,admin,2901396.0,2901396.0
4,Antalya,36.9081,30.6956,Turkey,TR,Antalya,admin,2426356.0,2426356.0
...,...,...,...,...,...,...,...,...,...
1600,Pülümür,39.4845,39.8953,Turkey,TR,Tunceli,minor,NaN,NaN
1601,Kırmızıköprü,39.3872,39.8061,Turkey,TR,Tunceli,minor,NaN,NaN
1602,Dağyolu,39.5402,39.9070,Turkey,TR,Tunceli,minor,NaN,NaN
1603,İbradı,37.0969,31.5992,Turkey,TR,Antalya,minor,NaN,NaN


In [7]:
citys_coordinates=citys_coordinates.loc[(citys_coordinates["capital"]=="admin")|(citys_coordinates["capital"]=="primary")]

In [8]:
citys_coordinates=citys_coordinates.rename(columns={"admin_name":"CITY"})
citys_coordinates

,city,lat,lng,country,iso2,CITY,capital,population,population_proper
0,Istanbul,41.0100,28.9603,Turkey,TR,İstanbul,admin,15154000.0,15029231.0
1,Ankara,39.9300,32.8500,Turkey,TR,Ankara,primary,5503985.0,5503985.0
2,İzmir,38.4127,27.1384,Turkey,TR,İzmir,admin,4320519.0,4320519.0
3,Bursa,40.1833,29.0667,Turkey,TR,Bursa,admin,2901396.0,2901396.0
4,Antalya,36.9081,30.6956,Turkey,TR,Antalya,admin,2426356.0,2426356.0
...,...,...,...,...,...,...,...,...,...
133,Gümüşhane,40.4597,39.4778,Turkey,TR,Gümüşhane,admin,57269.0,57269.0
134,Muğla,37.2167,28.3667,Turkey,TR,Muğla,admin,56619.0,56619.0
147,Ardahan,41.1167,42.7000,Turkey,TR,Ardahan,admin,42226.0,42226.0
151,Tunceli,39.1061,39.5481,Turkey,TR,Tunceli,admin,38504.0,38504.0


In [9]:
covid_19trdata1=pd.merge(covid_19trdata,citys_coordinates,how="left")

## The last updated population numbers are in the POPULATION column. (the one with the upper case letter)

In [10]:
covid_19trdata1

,ID,DATE_,SEQID,CITY,CITY2,_1DOSE,_2DOSE,_TOTAL,POPULATION,DIFF_1DOSE,...,PREVID,percent_vaccinated,city,lat,lng,country,iso2,capital,population,population_proper
0,438102,2021-06-26 22:41:31,5412,Adana,Adana,784464,355404,1139868,2258718,15,...,438021,0.504653,Adana,37.0000,35.3250,Turkey,TR,admin,2220125.0,2220125.0
1,438103,2021-06-26 22:41:31,5412,Adıyaman,Adiyaman,170847,77955,248802,632459,2,...,438022,0.393388,Adıyaman,37.7644,38.2763,Turkey,TR,admin,304615.0,304615.0
2,438104,2021-06-26 22:41:31,5412,Afyonkarahisar,Afyon,292611,138934,431545,736912,8,...,438023,0.585613,Afyonkarahisar,38.7581,30.5386,Turkey,TR,admin,299673.0,299673.0
3,438105,2021-06-26 22:41:31,5412,Ağrı,Agri,94807,41146,135953,535435,0,...,438024,0.253911,Ağrı,39.7225,43.0544,Turkey,TR,admin,149188.0,149188.0
4,438106,2021-06-26 22:41:31,5412,Aksaray,Aksaray,131889,62603,194492,423011,3,...,438025,0.459780,Aksaray,38.3686,34.0297,Turkey,TR,admin,295351.0,295351.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,438178,2021-06-26 22:41:31,5412,Uşak,Usak,169159,81055,250214,369433,6,...,438097,0.677292,Uşak,38.6833,29.4000,Turkey,TR,admin,252044.0,252044.0
77,438179,2021-06-26 22:41:31,5412,Van,Van,236641,91208,327849,1149342,5,...,438098,0.285249,Van,38.5019,43.4167,Turkey,TR,admin,353419.0,353419.0
78,438180,2021-06-26 22:41:31,5412,Yalova,Yalova,129888,65920,195808,276050,2,...,438099,0.709321,Yalova,40.6556,29.2750,Turkey,TR,admin,144407.0,144407.0
79,438181,2021-06-26 22:41:31,5412,Yozgat,Yozgat,151825,81924,233749,419095,1,...,438100,0.557747,Yozgat,39.8208,34.8083,Turkey,TR,admin,105167.0,105167.0


In [11]:
total_percantage_tr=covid_19trdata1["_TOTAL"].sum()/covid_19trdata1["POPULATION"].sum()
print("Total percentage vaccinated in Turkey: {}%".format(round(total_percantage_tr*100,2)))

Total percentage vaccinated in Turkey: 56.13%


In [12]:
map=folium.Map(location=[39.925533,32.866287],tiles="cartodbpositron",zoom_start=4)

mc=MarkerCluster()
for idx,row in covid_19trdata1.iterrows():
    if not math.isnan(row["lng"]) and not math.isnan(row["lat"]):
        mc.add_child(Marker(location=[row["lat"],row["lng"]],tooltip=str(round(row["percent_vaccinated"]*100,2))+"%"))

map.add_child(mc)

map